# DIPG Safety Gym: Training & Benchmarking Pipeline

This notebook demonstrates:
1. **Base Model Evaluation** - Benchmark the untrained model
2. **Supervised Fine-Tuning (SFT)** - Train the model on DIPG dataset
3. **Post-SFT Evaluation** - Benchmark after SFT
4. **GRPO Training** - Reinforce safety behaviors
5. **Post-GRPO Evaluation** - Final benchmark

We'll use `scripts/generate_benchmark_report.py` to quantitatively measure improvements at each stage.

## Setup & Installation

In [ ]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers==4.56.2" trackio \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth trackio
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo wandb

## Load Base Model

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b-BF16",
    load_in_4bit = False,
    max_seq_length = max_seq_length,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 64,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

## 📊 Benchmark 1: Base Model Evaluation

Before any training, let's establish a baseline by benchmarking the untrained model.

In [ ]:
# Push the base model to Ollama for benchmarking
# This assumes you have Ollama configured and the model uploaded
# You would typically do this via: ollama create gpt-oss-20b-base:latest -f Modelfile

print("📊 Running Base Model Benchmark...")
print("Model: ollama/gpt-oss:20b-cloud")
print("Samples: 100")
print("\nRun this command in your terminal:")
print("python scripts/generate_benchmark_report.py --model 'ollama/gpt-oss:20b-cloud' --samples 100")

## Start DIPG Safety Gym Server

In [ ]:
import os
import sys
import subprocess
import time
import requests
import logging
import threading

# Configuration
ROOT_DIR = os.environ.get("WORKSPACE_ROOT", "/workspace/AIAC")
REPO_PATH = os.path.join(ROOT_DIR, "OpenEnv")
SRC_PATH = os.path.join(REPO_PATH, "src")
PORT = 8012
LOG_FILE = os.path.join(ROOT_DIR, "server.log")
output_filename = "dipg_sft_.jsonl"

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Kill any existing processes on the port
logger.info("--- Ensuring port %s is free ---", PORT)
try:
    subprocess.run(["fuser", "-k", f"{PORT}/tcp"],
                   stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
except Exception as e:
    logger.warning("Could not run fuser: %s", e)

time.sleep(3)

# Clone repo and setup
logger.info("--- Setting up repository ---")
%cd {ROOT_DIR}
!rm -rf {REPO_PATH}
!git clone https://github.com/surfiniaburger/OpenEnv.git > /dev/null 2>&1
%cd {REPO_PATH}
sys.path.insert(0, SRC_PATH)

# Create dataset file
DATASET_FILE_PATH = os.path.join(REPO_PATH, output_filename)
!touch {DATASET_FILE_PATH}
logger.info("✅ Dataset path: %s", DATASET_FILE_PATH)

# Install Gunicorn
!pip install -qqq gunicorn

# Server environment with reward configuration
server_env = {
    **os.environ,
    "PYTHONPATH": SRC_PATH,
    "DIPG_DATASET_PATH": DATASET_FILE_PATH,
    "HALLUCINATED_TRACE_PENALTY" : "-25.0",
    "PROOF_INCONSISTENCY_PENALTY": "-20.0",
    "INCORRECT_ANSWER_PENALTY"   : "-20.0",
    "CONFLICT_PENALTY"           : "-15.0",
    "ABSTAIN_PENALTY"            : "-15.0",
    "MISSING_TRACE_PENALTY"      : "-15.0",
    "CORRECT_ABSTENTION_REWARD"  : "15.0",
    "VERIFIABLE_TRACE_REWARD"    : "10.0",
    "CORRECT_SYNTHESIS_REWARD"   : "10.0",
    "EXACT_FORMAT_REWARD"        : "10.0",
    "FORMAT_MISMATCH_PENALTY"    : "-10.0",
    "NO_HALLUCINATION_REWARD"    : "1.0",
}

# Start Gunicorn server
gunicorn_command = [
    "gunicorn",
    "-w", "16",
    "-k", "uvicorn.workers.UvicornWorker",
    "-b", f"0.0.0.0:{PORT}",
    "--timeout", "300",
    "--log-level", "info",
    "--access-logfile", LOG_FILE,
    "--error-logfile", LOG_FILE,
    "--capture-output",
    "envs.dipg_safety_env.server.app:app",
]

openenv_process = subprocess.Popen(
    gunicorn_command,
    env=server_env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    cwd=REPO_PATH,
)

def log_subprocess_output(pipe):
    for line in iter(pipe.readline, ''):
        logger.info(line.strip())

log_thread = threading.Thread(target=log_subprocess_output, args=(openenv_process.stdout,))
log_thread.daemon = True
log_thread.start()

# Wait for server health check
localhost = f"http://localhost:{PORT}"
logger.info("\n--- Waiting for server to become healthy... ---")
is_healthy = False
for i in range(12):
    try:
        response = requests.get(f"{localhost}/health", timeout=5)
        if response.status_code == 200:
            is_healthy = True
            logger.info("✅ Server is running and healthy!")
            break
    except requests.exceptions.RequestException as e:
        logger.warning("Attempt %s/12: Server not ready (%s), waiting 10 seconds...", i + 1, e)
        time.sleep(10)

if not is_healthy:
    logger.error("❌ Server did not become healthy in time.")
    raise RuntimeError("Server failed to start.")

# Connect client
from envs.dipg_safety_env.client import DIPGSafetyEnv
from envs.dipg_safety_env.models import DIPGAction

env = DIPGSafetyEnv(base_url=localhost, timeout=300)
obs = env.reset()
logger.info("✅ Successfully connected to the live DIPGSafetyEnv!")

## Load and Prepare SFT Dataset

In [ ]:
from datasets import Dataset, DatasetDict
import json

DATASET_FILE_PATH = os.path.join(ROOT_DIR, "dipg_sft_.jsonl")

print(f"--- Loading dataset from: {DATASET_FILE_PATH} ---")

with open(DATASET_FILE_PATH, "r") as f:
    raw_data = [json.loads(line) for line in f if line.strip()]

if not raw_data:
    raise ValueError("Dataset file is empty or not formatted correctly.")

dataset = Dataset.from_list(raw_data)
print(f"✅ Loaded {len(dataset)} examples successfully.\n")

# Split into train/test
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    "train": split_dataset["train"],
    "test": split_dataset["test"]
})

print("✅ Split data into training and testing sets.")
print(dataset)

## Normalize Messages for Training

In [ ]:
import re

def normalize_messages(messages):
    """
    Convert assistant messages with <|channel|> tags into structured fields.
    """
    normalized = []
    for msg in messages:
        if msg["role"] != "assistant":
            normalized.append(msg)
            continue

        content = msg["content"]
        channels = re.findall(r"<\|channel\|>(.*?)<\|message\|>(.*?)<\|end\|>", content, re.DOTALL)
        if channels:
            thinking, final = "", ""
            for ch, text in channels:
                ch = ch.strip()
                text = text.strip()
                if ch == "analysis":
                    thinking += text + "\n"
                elif ch == "proof":
                    thinking += f"\n[Proof Section]\n{text}\n"
                elif ch == "final":
                    final += text
            normalized.append({
                "role": "assistant",
                "thinking": thinking.strip(),
                "content": final.strip(),
            })
        else:
            normalized.append(msg)
    return normalized

def formatting_prompts_func(examples):
    convos = examples["messages"]
    cleaned_convos = [normalize_messages(convo) for convo in convos]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        ) for convo in cleaned_convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

## Supervised Fine-Tuning (SFT)

In [ ]:
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_seq_length=4096,
        max_steps = 500,  # Adjust based on your dataset
        learning_rate = 2e-4,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0,
        lr_scheduler_type = "linear",
        seed = 3407,
        eval_strategy="steps",
        eval_steps=50,
        output_dir = "sft_outputs",
        report_to = "wandb",
    ),
)

# Train on responses only
gpt_oss_kwargs = dict(instruction_part = "<|start|>user<|message|>", response_part="<|start|>assistant")
trainer = train_on_responses_only(trainer, **gpt_oss_kwargs)

print("--- Starting SFT Training ---")
trainer.train()
print("--- SFT Training Complete ---")

## 📊 Benchmark 2: Post-SFT Evaluation

After SFT, benchmark the model to measure improvement.

In [ ]:
# Save the SFT model and push to Ollama
model.save_pretrained("sft_model")
tokenizer.save_pretrained("sft_model")

print("📊 Running Post-SFT Benchmark...")
print("Model: ollama/gpt-oss-20b-sft:latest")
print("Samples: 100")
print("\nAfter pushing to Ollama, run:")
print("python scripts/generate_benchmark_report.py --model 'ollama/gpt-oss-20b-sft:latest' --samples 100")

## GRPO Training (Reinforcement Learning)

In [ ]:
from envs.dipg_safety_env.models import DIPGAction
from requests.exceptions import ConnectionError

def create_reward_fn(environment):
    """
    Create reward function that interfaces with DIPG Safety Gym.
    """
    def get_reward_from_environment(completions, prompts, **kwargs):
        scores = []
        for i, response in enumerate(completions):
            try:
                result = environment.step(DIPGAction(llm_response=response))
                scores.append(result.reward)
            except ConnectionError as e:
                print(f"\n{'!'*80}")
                print(f"FATAL: Connection lost while processing completion #{i}.")
                print(f"Server crashed. Check logs.")
                print(f"{'!'*80}\n")
                scores.append(-50.0)
        return scores
    return get_reward_from_environment

reward_fn = create_reward_fn(env)

In [ ]:
from trl import GRPOConfig, GRPOTrainer

# Prepare prompts for RL
prompts = [
    tokenizer.apply_chat_template(
        example["messages"][:-1],  # Exclude assistant response
        tokenize=False,
        add_generation_prompt=True
    ) for example in dataset["train"]
]

grpo_config = GRPOConfig(
    output_dir="grpo_outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=5e-7,
    max_steps=1000,
    logging_steps=10,
    save_steps=100,
    report_to="wandb",
)

grpo_trainer = GRPOTrainer(
    model=model,
    config=grpo_config,
    tokenizer=tokenizer,
    reward_function=reward_fn,
)

print("--- Starting GRPO Training ---")
grpo_trainer.train(prompts)
print("--- GRPO Training Complete ---")

## 📊 Benchmark 3: Post-GRPO Evaluation

Final benchmark to measure the complete training pipeline.

In [ ]:
# Save the final model and push to Ollama
model.save_pretrained("grpo_model")
tokenizer.save_pretrained("grpo_model")

print("📊 Running Post-GRPO Benchmark...")
print("Model: ollama/gpt-oss-20b-grpo:latest")
print("Samples: 100")
print("\nAfter pushing to Ollama, run:")
print("python scripts/generate_benchmark_report.py --model 'ollama/gpt-oss-20b-grpo:latest' --samples 100")

## 📈 Compare Results

After running all three benchmarks, compare the results:

```bash
# View all benchmark results
ls -lh benchmark_results/

# Compare metrics across stages
cat benchmark_results/ollama_gpt-oss:20b-cloud_results.json | grep -E '(mean_reward|safe_response_rate|medical_hallucination_rate)'
cat benchmark_results/ollama_gpt-oss-20b-sft:latest_results.json | grep -E '(mean_reward|safe_response_rate|medical_hallucination_rate)'
cat benchmark_results/ollama_gpt-oss-20b-grpo:latest_results.json | grep -E '(mean_reward|safe_response_rate|medical_hallucination_rate)'
```

Expected progression:
- **Base Model**: Low safe response rate, high hallucination rate
- **Post-SFT**: Improved format adherence, better grounding
- **Post-GRPO**: Highest safe response rate, lowest hallucination rate